In [15]:
import os
import json
from dotenv import load_dotenv

from langchain.agents import load_tools, initialize_agent, AgentType
from langchain_community.llms import Ollama

from common_functions import ensure_llama_running


ensure_llama_running()
load_dotenv()

llm_model = os.getenv('LLM_MODEL')
llm = Ollama(model=llm_model)
llm.get_name()

'Ollama'

In [16]:
tools = load_tools([
    "requests_all", "llm-math", "openweathermap-api", "google-serper"
], llm=llm)  # "wikipedia", "google-search"

# from langchain import hub
# print("Pulling prompt...", end=" ", flush=True)
# prompt = hub.pull("hwchase17/structured-chat-agent")
# print("done")
# agent = create_structured_chat_agent(
# 	tools,
# 	llm,
# 	prompt=prompt,
# 	# agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # uses Chain-of-Thought
# 	# verbose=True,
# 	# handle_parsing_errors=True,
# )
# agent_executor = AgentExecutor(
#     agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
# )

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # uses Chain-of-Thought
	verbose=True,
	handle_parsing_errors=True,
)

def ask_query(query):
	response = agent.invoke({"input": query})
	if isinstance(response, dict):
		response = response.get('output')
	response = response.strip()
	if '{' in response[:15]:
		# get text from first { to last }
		response = response[response.index('{'):response.rindex('}')+1]
	if response[0] == '{':
		response_json = json.loads(response)
		answer = response_json.get('action_input')
		return answer
	return response

In [17]:
ask_query("Who is the current president of France?")



> Entering new AgentExecutor chain...
Action:
```
{
   "action": "Final Answer",
   "action_input": "Emmanuel Macron"
}
```


> Finished chain.


'Emmanuel Macron'

In [10]:
ask_query("What is weather of the capital of France?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "open_weather_map",
  "action_input": "Paris,FR"
}
```

Observation: In Paris,FR, the current weather is as follows:
Detailed status: clear sky
Wind speed: 2.57 m/s, direction: 60°
Humidity: 82%
Temperature: 
  - Current: 15.33°C
  - High: 16.88°C
  - Low: 14.06°C
  - Feels like: 15.06°C
Rain: {}
Heat index: None
Cloud cover: 0%
Thought:Action:
```{
    "action": "Final Answer",
    "action_input": "The current weather in Paris, FR is a clear sky with wind speed of 2.57 m/s from the direction of 60°, humidity of 82%, and temperature ranging from 14.06°C to 16.88°C."
}```


> Finished chain.


'The current weather in Paris, FR is a clear sky with wind speed of 2.57 m/s from the direction of 60°, humidity of 82%, and temperature ranging from 14.06°C to 16.88°C.'

In [5]:
ask_query("Latest news")



> Entering new AgentExecutor chain...
Action:
{
   "action": "google_search",
   "action_input": "latest news"
}



> Finished chain.


'Action:\n{\n   "action": "google_search",\n   "action_input": "latest news"\n}'

In [6]:
ask_query("What happened in November 2023?")



> Entering new AgentExecutor chain...
Action:
{
   "action": "google_search",
   "action_input": "What events occurred on November 2023"
}



> Finished chain.


'Action:\n{\n   "action": "google_search",\n   "action_input": "What events occurred on November 2023"\n}'

In [33]:
ask_query("On which date was George Washington born?")



> Entering new AgentExecutor chain...
Thought:
Action:
```
{
   "action": "Final Answer",
   "action_input": "February 22, 1732"
}
```


> Finished chain.


'February 22, 1732'